# Darts benchamrk 

This notebook showcases tools developed for benchmarking the different forecasting models in Darts, which could be useful for other developers.
We will showcase:

**Auto-ML with Optuna and raytune**: hyperparameter tuning with predefined broad parameter space for all model supported<br />
**Experiment**: A function to perform cross comparison of a list of models on a list of datasets

In [ ]:
#@title Install dependencies
!pip install -q git+https://github.com/Loudegaste/darts-benchmark

from darts_benchmark.benchmark_tools import Dataset, experiment, silence_prompt
from darts.models import NaiveSeasonal, NHiTSModel, Prophet
from darts.utils import missing_values
from darts_benchmark.optuna_search import optuna_search
from darts.datasets import AirPassengersDataset, WeatherDataset

models = [NaiveSeasonal, NHiTSModel, Prophet]

### Dataset loading
To ensure a uniform structure for the data loading, each dataset must be contained in a ```benchmark_tools.Dataset``` Named tuple with the following fields:

**name**: Dataset name for display<br />
**series**: A darts.timeseries object<br />
**future_covariates**:(Optional)<br />
**past_covariates**:(Optional)<br />



In [7]:
#@title Load two datasets

air_pass_series = missing_values.fill_missing_values(AirPassengersDataset().load())
weather_ds = missing_values.fill_missing_values(WeatherDataset().load().resample("1h"))[
    -1500:
]
weather_past_cov = weather_ds[
    ["p (mbar)", "wv (m/s)", "wd (deg)", "rain (mm)", "raining (s)", "SWDR (W/m²)"]
]
weather_series = weather_ds["T (degC)"]

dataset_air_passengers = Dataset(name="Air passengers", series=air_pass_series)
dataset_weather = Dataset(
    name="Weather", series=weather_series, past_covariates=weather_past_cov
)

In [ ]:
#@title Load two datasets

silence_prompt()
best_params = optuna_search(
    model_class=Prophet, **dataset_air_passengers._asdict(), time_budget=60
)

# display the output after hyperparameter search
model = Prophet(**best_params)
forecasts = model.historical_forecasts(series=dataset_air_passengers.series, start=0.8)
dataset_air_passengers.series.plot(label="actual")
forecasts.plot(label="prediction")

In [ ]:
results = experiment(
    list_datasets=[dataset_air_passengers, dataset_weather],
    models=models,
    grid_search=False,
    forecast_horizon=0.05, # the forecast horizon will be set to 5% of the test set length
    repeat=3,
    silent_search=True,
)

print("\n\n", results)